In [1]:
import cv2
import numpy as np
from PIL import Image
import rasterio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# import DeepLab
from model import DeepLabModel, DeepLab

In [2]:
# input video file path
video = str('./input/video/DJI_0008_short.mp4')

# output foto file path
foto = str('./output/foto/test1.png')
foto_jpg = str('./output/foto/test1_umgewandelt.jpg')


# output video file path
output_video = str('./output/video/test_output_null.avi') # Path to Output-Video

# Frames per Second for Output Video File
fps = float(30.0)

In [ ]:
# Load .png image
image = cv2.imread('./output/foto/test1.png')

# Save .jpg image
cv2.imwrite('./output/foto/test1_umgewandelt.jpg', image, [int(cv2.IMWRITE_JPEG_QUALITY), 100])

In [ ]:
foto_jpg

In [ ]:
a = np.array(df_train.iloc[0].Image.split(),dtype='float')
a = a.reshape(96,96)
plt.imshow(a)

In [ ]:
fig = plt.figure()
ax = fig.subplots()
ax.imshow(foto_jpg)

In [ ]:
from osgeo import gdal
from osgeo.gdalconst import GA_Update
filename = 'somefile.tif'
nodata = 0
# open the file for editing
ras = gdal.Open(filename, GA_Update)
# loop through the image bands
for i in range(1, ras.RasterCount + 1):
    # set the nodata value of the band
    ras.GetRasterBand(i).SetNoDataValue(nodata)
# unlink the file object and save the results
ras = None

In [3]:
cap = cv2.VideoCapture(video)


cap.set(cv2.CAP_PROP_POS_AVI_RATIO,0)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
videoFPS = int(cap.get(cv2.CAP_PROP_FPS))

print (f"frameCount: {frameCount}")
print (f"frameWidth: {frameWidth}")
print (f"frameHeight: {frameHeight}")
print (f"videoFPS: {videoFPS}")

buf = np.empty((
    frameCount,
    frameHeight,
    frameWidth,
    3), np.dtype('uint8'))

fc = 0
ret = True

while (fc < frameCount):
    ret, buf[fc] = cap.read()
    fc += 1

cap.release()
videoArray = buf

print (f"DURATION: {frameCount/videoFPS}")

frameCount: 44
frameWidth: 3840
frameHeight: 2160
videoFPS: 29
DURATION: 1.5172413793103448


In [4]:
# get dimension of video input
width_input  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   # width`
height_input = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  #  `height`

In [5]:
# initialize video save optionen
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_a = cv2.VideoWriter(output_video,fourcc, fps, (width_input,height_input)) # Output of whole frame

# extract an safe video frames
vidcap = cv2.VideoCapture(video)
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

In [6]:
print("Init successful!")

Init successful!


In [10]:
#deeplab definitions

def create__label_colormap():
     return np.asarray([
         [0, 0, 0],
         [0, 150, 0],
         [255, 150, 150],
     ])
    

def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
    label: A 2D array with integer type, storing the segmentation label.

    Returns:
    result: A 2D array with floating type. The element of the array
      is the color indexed by the corresponding element in the input label
      to the PASCAL color map.

    Raises:
    ValueError: If label is not of rank 2 or its value is larger than color
      map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create__label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

In [11]:
# Set basic variables
current_frame = 0
id = 1

In [ ]:
ret, first_frame = cap.read()
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

In [12]:
# -------1--------
model = DeepLabModel('model/frozen_inference_graph_cars.pb')

cap = cv2.VideoCapture(video)
cap.set(cv2.CAP_PROP_POS_AVI_RATIO,0)

img_array = []

while True:
    current_frame += 1
    ret, frame = cap.read()
    
    if frame is None:
        print('finished!')
        break
        
    mask = model.run(Image.fromarray(frame))       
    img_array.append(label_to_color_image(mask).astype(np.uint8))
    



finished!


In [13]:
#----2------
size = img_array[0].shape[:2][::-1]

In [14]:
#------3------
out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [ ]:
label_to_color_image(mask).astype(np.uint8).shape

In [ ]:
size

In [ ]:
type(first_frame)

In [ ]:
current_frame

In [ ]:
buf[0,:,:,:]

In [ ]:
original_im = Image.open(file)

In [ ]:
model = DeepLabModel('model/frozen_inference_graph_cars.pb')

In [ ]:
Image.fromarray(label_to_color_image(mask).astype(np.uint8)).save('output/test1.png')

---------

In [ ]:
mask = model.run(Image.fromarray(buf[94,:,:,:]))

In [ ]:
Image.fromarray(label_to_color_image(mask).astype(np.uint8))